In [1]:
import pandas as pd
from config.constants import EXCEL_EXPORT_PATH, LEAGUES_DATA_DICT, SOLOQ

In [2]:
df = pd.read_excel(LEAGUES_DATA_DICT[SOLOQ][EXCEL_EXPORT_PATH])

In [4]:
mad_df = df.loc[df.team_abbv == 'MAD']

## How many times they played a champion

In [6]:
champs_practiced = mad_df.groupby(['player_name', 'champ_name']).count()['currentAccountId'].to_frame().reset_index()

In [7]:
champs_practiced.rename(columns={'currentAccountId': 'times_played'}, inplace=True)

In [8]:
df2 = champs_practiced

### What champions they did have to practice

In [10]:
dict0 = {'Werlyb': ['Garen', 'Tryndamere', 'Kalista', 'Jax'], 
         'Selfmade': ['Fiddle', 'Hecarim', 'Lee Sin'], 
         'Nemesis': ['Sejuani', 'Caitlyn', 'Soraka', 'Rakan'],
         'Crownshot': ['Ornn', 'Yasuo', 'Alistar'],
         'Falco': ['Fiora', 'Malzahar', 'Nautilus']}

In [11]:
practice_champs = pd.DataFrame([(key, v, 1) for key, values in dict0.items() for v in values]).rename(columns={0: 'player_name', 1: 'champ_name', 2: 'should_train'})

In [12]:
practiced_champs = df2.merge(practice_champs, on=['player_name', 'champ_name'], how='outer').sort_values(['player_name', 'champ_name']).fillna(0)

In [13]:
practiced_champs[['times_played', 'should_train']] = practiced_champs[['times_played', 'should_train']].astype(int)

## How many times they played together

In [14]:
dict1 = {}
for name in champs_practiced.player_name.unique():
    df = champs_practiced.loc[(champs_practiced.player_name == name)]
    dict1[name] = {row[1][1]: row[1][2] for row in df.iterrows()}

In [15]:
together = mad_df.groupby(['gameId', 'queueId', 'player_name'], as_index=False).count()

In [16]:
duplicated_ids = together.duplicated('gameId', keep=False)

In [17]:
played_together = together.ix[duplicated_ids]

C:\Users\david\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


In [18]:
from collections import defaultdict

In [19]:
dict2 = defaultdict(int)

In [20]:
for gameid in played_together.gameId.unique():
    df = played_together.loc[played_together.gameId == gameid]
    names = list(df.sort_values('player_name').player_name)
    if len(names) == 2:
        team_name = names[0] + '_' + names[1]
    elif len(names) == 3:
        team_name = names[0] + '_' + names[1] + '_' + names[2]
    elif len(names) == 5:
        team_name = names[0] + '_' + names[1] + '_' + names[2] + '_' + names[3] + '_' + names[4]
    dict2[team_name] += 1

In [21]:
df3 = pd.DataFrame(dict(dict2), index=(0,)).T.rename(columns={0: 'times_played_together'})

## To excel

In [23]:
writer = pd.ExcelWriter('../exports/solo_queue_report.xlsx')
practiced_champs.to_excel(writer,'Champions practiced')
df3.to_excel(writer, 'Times played together')
writer.save()